In [1]:
import pandas as pd
import numpy as np

import os

from mrmr import mrmr_classif

# Load Data

In [2]:
BASEDIR = '../data'

metadata = pd.read_csv(
    os.path.join(BASEDIR, 'metadata', 'metadata.csv')
)

tax_genus = pd.read_csv(
    os.path.join(BASEDIR, 'taxonomy', 'taxonomy_relabd.genus.csv'),
)

pt_ra_1e_1 = pd.read_csv(
    os.path.join(BASEDIR, 'phylotypes', 'phylotype_relabd.1e_1.csv'),
)

sv_counts = pd.read_csv(
    os.path.join(BASEDIR, 'sv_counts', 'sp_sv_long.csv'),
)

meta = metadata[['specimen', 'participant_id', 'collect_wk']]

## sv_counts for features

In [3]:
def sv_to_num(sv):
    try:
        num = int(sv.split('___')[1])
    except:
        num = None
    return num

def sv_for_features(df, index, cols):
    df['num_sv'] = df.sv.map(lambda x: sv_to_num(x))
    
    num_features = 53129
   
    features = []
    for specimen in index:
        feature = np.zeros(num_features)
        new_df = df[df['specimen'] == specimen]
        sv = np.array(new_df['num_sv'])
        fract = np.array(new_df['fract'])
        for i in range(len(sv)):
            if sv[i] == None:
                continue
            else:
                feature[sv[i]-1] = fract[i]
        features.append(feature)
    df_features = pd.DataFrame(features, index=index, columns = cols)
    df_features.index.name = 'specimen'
    return df_features

index = np.array(metadata.specimen)
sv_cols = np.load('../sv_cols.npy') # sorted sv array

new_sv = sv_for_features(sv_counts, index, sv_cols)

## y_labels

In [4]:
y_df = metadata[['was_preterm', 'participant_id']]
y_df = y_df.drop_duplicates(['participant_id'], keep='first')
y_df = y_df.set_index('participant_id')

In [5]:
y_df

,was_preterm
participant_id,
A00001,False
A00002,False
A00003,False
A00004,False
A00005,False
...,...
J00111,False
J00112,False
J00113,False


# MRMR feature selection

In [6]:
meta = metadata[['specimen', 'participant_id', 'collect_wk']]

def get_mrmr(df, num = 300):
    df = pd.merge(meta, df, on='specimen')
    df = df[df['collect_wk'] <= 32] # For preterm
    dup = df.duplicated('participant_id', keep=False)
    duplicated = df[dup]
    dup_mean = duplicated.groupby(['participant_id'], as_index=False).aggregate(np.mean)
    df = df.drop_duplicates(['participant_id'], keep=False)
    df = pd.concat([df, dup_mean])
    df = df.sort_values(by=['participant_id'], ascending=[True]) 
    df = df.set_index('participant_id')
    df = df.drop(columns = ['specimen', 'collect_wk'])
    
    selected_features = mrmr_classif(X=df, y=y_df.loc[df.index], K=num)
    
    return selected_features

In [7]:
tax_selected_features = get_mrmr(tax_genus)
tax_selected_features

100%|██████████| 300/300 [01:47<00:00,  2.79it/s]


['Prevotella',
 'Ureaplasma',
 'Jeotgalicoccus',
 'Atopobiaceae/Coriobacteriaceae',
 'Chryseobacterium',
 'Pelomonas',
 'Enterobacteriaceae',
 'Haemophilus',
 'Yokenella',
 'Citrobacter/Kluyvera',
 'Dorea/Butyrivibrio',
 'Streptococcus',
 'Desulfatiglans',
 'Pseudomonadales/Acidimicrobiales',
 'Lactobacillales',
 'Cyanobacteria',
 'Candidatus Paracaedibacter',
 'Cryptobacterium',
 'Holosporaceae/Rickettsiaceae',
 'Pseudobutyrivibrio/Butyrivibrio',
 'Alcanivorax',
 'Corynebacteriaceae/Lachnospiraceae',
 'Rhodobacteraceae/Clostridiales_Family_XIII._Incertae_Sedis',
 'Psychrobacter',
 'Pseudoramibacter',
 'Tardiphaga',
 'Mesorhizobium',
 'Bacteroides/Prevotella',
 'Geodermatophilus',
 'Luteitalea',
 'Spirochaetaceae/Desulfobacteraceae',
 'Ciceribacter',
 'Agaricicola',
 'Mycoplasmataceae/Hungateiclostridiaceae',
 'Alphaproteobacteria/Erysipelotrichia',
 'Marinobacter',
 'Phyllobacterium',
 'Flavobacterium/Oligella',
 'Atopobium/Fusobacterium',
 'Deinococci/Acidobacteriia',
 'Spirochaetace

In [8]:
pt_selected_features = get_mrmr(pt_ra_1e_1)
pt_selected_features

100%|██████████| 300/300 [08:21<00:00,  1.67s/it]


['pt__00019',
 'pt__04975',
 'pt__00371',
 'pt__07247',
 'pt__09501',
 'pt__00702',
 'pt__00920',
 'pt__08646',
 'pt__00903',
 'pt__01216',
 'pt__00838',
 'pt__02916',
 'pt__00135',
 'pt__00036',
 'pt__00589',
 'pt__01556',
 'pt__04090',
 'pt__06530',
 'pt__08014',
 'pt__03486',
 'pt__01471',
 'pt__00986',
 'pt__01321',
 'pt__07701',
 'pt__01489',
 'pt__02445',
 'pt__06816',
 'pt__01600',
 'pt__00889',
 'pt__07723',
 'pt__00342',
 'pt__01081',
 'pt__01474',
 'pt__01431',
 'pt__00981',
 'pt__00178',
 'pt__00987',
 'pt__03566',
 'pt__00532',
 'pt__00201',
 'pt__00583',
 'pt__09505',
 'pt__09016',
 'pt__01376',
 'pt__08467',
 'pt__03241',
 'pt__02466',
 'pt__08818',
 'pt__01935',
 'pt__06720',
 'pt__00433',
 'pt__06508',
 'pt__06674',
 'pt__01798',
 'pt__04820',
 'pt__06776',
 'pt__08366',
 'pt__05507',
 'pt__06309',
 'pt__01270',
 'pt__08517',
 'pt__03039',
 'pt__01909',
 'pt__03791',
 'pt__07551',
 'pt__01936',
 'pt__01901',
 'pt__01088',
 'pt__01240',
 'pt__03223',
 'pt__00482',
 'pt__

In [9]:
sv_selected_features = get_mrmr(new_sv)
sv_selected_features

100%|██████████| 300/300 [38:59<00:00,  7.80s/it]


['combinedsv___33040',
 'combinedsv___32987',
 'combinedsv___36074',
 'combinedsv___35780',
 'combinedsv___34460',
 'combinedsv___35463',
 'combinedsv___44564',
 'combinedsv___33107',
 'combinedsv___28929',
 'combinedsv___6990',
 'combinedsv___1716',
 'combinedsv___34614',
 'combinedsv___26244',
 'combinedsv___19868',
 'combinedsv___3556',
 'combinedsv___33615',
 'combinedsv___35539',
 'combinedsv___17753',
 'combinedsv___35434',
 'combinedsv___33483',
 'combinedsv___33226',
 'combinedsv___45226',
 'combinedsv___32420',
 'combinedsv___51675',
 'combinedsv___20132',
 'combinedsv___51997',
 'combinedsv___32665',
 'combinedsv___7722',
 'combinedsv___6830',
 'combinedsv___20322',
 'combinedsv___51074',
 'combinedsv___40870',
 'combinedsv___50676',
 'combinedsv___13060',
 'combinedsv___20425',
 'combinedsv___36117',
 'combinedsv___30212',
 'combinedsv___20683',
 'combinedsv___34318',
 'combinedsv___8512',
 'combinedsv___6484',
 'combinedsv___52933',
 'combinedsv___35357',
 'combinedsv___233

In [10]:
all_features = np.array(tax_selected_features + pt_selected_features + sv_selected_features)
np.save('selected_features.npy', all_features)